# Check with Kelly:
## Axes values look correct, used a negative sign to invert the major axis (i think), just need to make sure they're really ligned up correctly. 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from PIL import Image



import sys 

sys.path.insert(1, 'C:/Users/boibr/OneDrive/Documents/GitHub/MaNGA-VelMap/')
from GenerateCutout import get_cutout

In [4]:
data_folder = "/Users/Emlilio/School/Internships/UR ASTRO/Data/" #create a variable for directory to common folder

In [5]:
drpall_fn = data_folder + "drpall_ttype_R90.fits"
drpall = Table.read(drpall_fn, format="fits",hdu=1)

In [27]:
    
for i in range(len(drpall[:300])):
    plateifu = drpall['plateifu'][i]
    ra = drpall['objra'][i]
    dec = drpall['objdec'][i]
    r90 = drpall['R90'][i]
    directory = data_folder + 'Plots/Cutouts/'

    #axes code
    center_coords = SkyCoord(ra*u.deg,dec*u.deg)
    pos_angle = drpall['nsa_elpetro_phi'][i]          
    axis_ratio = drpall['nsa_sersic_ba'][i]

    #point 1
    point1 = center_coords.directional_offset_by(-(pos_angle)*u.deg,r90*u.arcsec)  
    alpha1 = point1.ra.deg
    delta1 = point1.dec.deg
    
    #point 2
    point2 = center_coords.directional_offset_by(-(pos_angle+180)*u.deg,r90*u.arcsec)
    alpha2 = point2.ra.deg
    delta2 = point2.dec.deg
    #point 3
    point3 = center_coords.directional_offset_by((pos_angle+90)*u.deg,(r90*axis_ratio)*u.arcsec)
    alpha3 = point3.ra.deg
    delta3 = point3.dec.deg
    
    #point 4
    point4 = center_coords.directional_offset_by((pos_angle+270)*u.deg,(r90*axis_ratio)*u.arcsec)
    alpha4 = point4.ra.deg
    delta4 = point4.dec.deg
    
    try:
        if (r90 == -999.0):
            get_cutout(plateifu, ra, dec, r90, data_folder + 'Plots/Bad_Cutouts/' )
            
        else:
            #reading image
            gal_image,wcs = get_cutout(plateifu, ra, dec, r90, directory)
            img = np.asarray(Image.open(data_folder + 'Plots/Cutouts/'+str(plateifu)+'.jpg'))
        
            #creating figure
            fig, ax = plt.subplots(subplot_kw=dict(projection=wcs), tight_layout = True, figsize=(7,5))
            
            #axes line code
            ax.plot([alpha1,alpha2],[delta1,delta2],'g',transform = ax.get_transform('world'))
            ax.plot([alpha3,alpha4],[delta3,delta4],'g',transform = ax.get_transform('world'))

            #other default stuff
            ax.imshow(img, origin='lower')
            ax.grid(color='white', ls='solid')
            ax.set(title = str(plateifu) ,xlabel='Declination', ylabel='Right Acsension')
            plt.savefig(data_folder + 'Plots/MajorMinorAxes_plots/'+plateifu+'_graph_axes')
            plt.close()

    
    except:
        continue